In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

In [2]:
model_id = "codellama/CodeLlama-7b-Instruct-hf"
# model_id = "Salesforce/instructcodet5p-16b"
# model_id = "Salesforce/codegen25-7b-mono"

tokenizer = AutoTokenizer.from_pretrained(model_id)

In [6]:
!pip install accelerate

Defaulting to user installation because normal site-packages is not writeable


In [4]:
pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    # device_map="auto",
)

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [8]:
code_snippet = """def svg_to_image(string, size=None):
    if isinstance(string, unicode):
        string = string.encode('utf-8')
    renderer = QtSvg.QSvgRenderer(QtCore.QByteArray(string))
    if not renderer.isValid():
        raise ValueError('Invalid SVG data.')
    if size is None:
        size = renderer.defaultSize()
    image = QtGui.QImage(size, QtGui.QImage.Format_ARGB32)
    painter = QtGui.QPainter(image)
    renderer.render(painter)
    return image"""

In [9]:
prompt = "Write a summary for the following code in natural language:\n"

In [10]:
sequences = pipeline(
    prompt + code_snippet,
    do_sample=True,
    temperature=0.3,
    top_p=0.3,
    num_return_sequences=1,
    # eos_token_id=tokenizer.eos_token_id,
    max_length=100,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/home/sir_timio/.local/lib/python3.10/site-packages/transformers/generation/utils.py:1281: UserWarning: Input length of input_ids is 165, but `max_length` is set to 100. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


Result: Write a summary for the following code in natural language:
def svg_to_image(string, size=None):
    if isinstance(string, unicode):
        string = string.encode('utf-8')
    renderer = QtSvg.QSvgRenderer(QtCore.QByteArray(string))
    if not renderer.isValid():
        raise ValueError('Invalid SVG data.')
    if size is None:
        size = renderer.defaultSize()
    image = QtGui.QImage(size, QtGui.QImage.Format_ARGB32)
    painter = QtGui.QPainter(image)
    renderer.render(painter)
    return image



In [13]:
code_snippet = """
def history_to_hash(npz: Union[None, str, FullGeneration]):
    if npz is None:
        return get_md5_hex(b"None")
    if isinstance(npz, str):
        return get_hash_from_voice_name(npz)
    npz_str = json.dumps(
        {
            "semantic_prompt": npz["semantic_prompt"].tolist(),
            "coarse_prompt": npz["coarse_prompt"].tolist(),
            "fine_prompt": npz["fine_prompt"].tolist(),
        }
    )
    npz_as_str = npz_str.encode("utf-8")
    return get_md5_hex(npz_as_str)
"""
sequences = pipeline(
    prompt + code_snippet,
    do_sample=True,
    temperature=0.3,
    top_p=0.3,
    num_return_sequences=1,
    # eos_token_id=tokenizer.eos_token_id,
    max_length=100,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Result: Write a summary for the following code in natural language:

def history_to_hash(npz: Union[None, str, FullGeneration]):
    if npz is None:
        return get_md5_hex(b"None")
    if isinstance(npz, str):
        return get_hash_from_voice_name(npz)
    npz_str = json.dumps(
        {
            "semantic_prompt": npz["semantic_prompt"].tolist(),
            "coarse_prompt": npz["coarse_prompt"].tolist(),
            "fine_prompt": npz["fine_prompt"].tolist(),
        }
    )
    npz_as_str = npz_str.encode("utf-8")
    return get_md5_hex(npz_as_str)




In [12]:
code_snippet = """
def fibonacci(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fibonacci(n-1) + fibonacci(n-2)
"""
sequences = pipeline(
    prompt + code_snippet,
    do_sample=True,
    temperature=0.3,
    top_p=0.3,
    num_return_sequences=1,
    # eos_token_id=tokenizer.eos_token_id,
    max_length=100,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Result: Write a summary for the following code in natural language:

def fibonacci(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fibonacci(n-1) + fibonacci(n-2)

Answer: This is a recursive function that calculates the nth Fibonacci number.

Answer: This is a recursive function


In [58]:
sequences = pipeline(
    prompt + code_snippet,
    do_sample=True,
    temperature=0.2,
    top_p=0.9,
    num_return_sequences=1,
    # eos_token_id=tokenizer.eos_token_id,
    max_length=100,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Result: Write a summary for the following code in natural language:
class CustomCallbackLLMChain(CustomCallbackChain):
    llm: BaseChatModel

    def retry_llm(self, messages, retry_count=5):
        sleep_duration = 0.5
        for _idx in range(retry_count - 1):
            try:
                return self.llm(messages)
            except Exception as e:
                self.fire_log(f"LLM failed with error: {e}; Retrying")
                time.sleep(sleep_duration)
            sleep_duration *= 2
        return self.llm(messages)

